## Replacing ligand parameters in an already-parametrized system

This example applies SMIRNOFF-format parameters to a BRD4 inhibitor from the [living review on binding free energy benchmark systems](https://www.annualreviews.org/doi/abs/10.1146/annurev-biophys-070816-033654) by Mobley and Gilson. The BRD4 system comes from the [accompanying GitHub repository](https://github.com/MobleyLab/benchmarksets/tree/master/input_files/BRD4).

This example uses [ParmEd](http://parmed.github.io) to take a protein-ligand system parameterized with an alternate force field, and replace the force field used for the ligand with an OpenFF force field. This example is meant to illustrate how to apply parameters to a single ligand, but it's also easy to process many ligands.

### Loading the already-parametrized system

In [ ]:
# Retrieve protein and ligand files for BRD4 and a docked inhibitor from the benchmark systems GitHub repository
# https://github.com/MobleyLab/benchmarksets
import requests
repo_url = 'https://raw.githubusercontent.com/MobleyLab/benchmarksets/master/input_files/'
sources = {
    'system.pdb' : repo_url + 'BRD4/prmtop-coords/BRD4-1.pdb', # complete system (protein+ligand+solvent+ions)
    'ligand.sdf'   : repo_url + 'BRD4/sdf/ligand-1.sdf', # ligand molecular identity
}
for (filename, url) in sources.items():
    r = requests.get(url)
    open(filename, 'w').write(r.text)

In [ ]:
# Read complete system in OpenMM PDBFile
from simtk.openmm.app import PDBFile
system_pdb = 'system.pdb'
pdbfile = PDBFile(system_pdb)

In [ ]:
# Load the definition of the small molecule in the system from an SDF file
from openforcefield.topology import Molecule
ligand = Molecule.from_file('ligand.sdf')

In [ ]:
# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P with compatible ions
from simtk.openmm import app
forcefield = app.ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml', 'amber/tip3p_HFE_multivalent.xml')
# Use the SMIRNOFF residue template generator to load the openff-1.0.0 ("Parsley") that knows about the ligand
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
smirnoff = SMIRNOFFTemplateGenerator(forcefield='openff-1.0.0', molecules=ligand)
# Register the SMIRNOFF template generator
forcefield.registerTemplateGenerator(smirnoff.generator)

In [ ]:
# Create a parameterized OpenMM System from the PDB topology without bond constraints so we can convert to other packages
from simtk import unit
system = forcefield.createSystem(pdbfile.topology, nonbondedMethod=app.PME, constraints=None, rigidWater=False, removeCMMotion=False)

###  Create a ParmEd Structure object to export to other formats

In [ ]:
# Create the complex Structure
import parmed
complex_structure = parmed.openmm.load_topology(pdbfile.topology, system=system)

# Copy over the original coordinates and box vectors
complex_structure.coordinates = orig_structure.coordinates
complex_structure.box_vectors = orig_structure.box_vectors

### Export to AMBER and GROMACS formats

We started off in AMBER format, and presumably may want to continue in that format -- so let's write out to AMBER and GROMACS format:

In [ ]:
# Export the Structure to AMBER files
complex_structure.save('complex.prmtop', overwrite=True)
complex_structure.save('complex.inpcrd', overwrite=True)

# Export the Structure to Gromacs files
complex_structure.save('complex.gro', overwrite=True)
complex_structure.save('complex.top', overwrite=True)

That should conclude our work in this example. However, perhaps we should just doublecheck by ensuring we can actually run some dynamics on the combined system without any trouble.


## As a test, run some dynamics on the combined system

First, we create an OpenMM system, as we've done in other examples here. We can do this, in this case, using ParmEd's built-in `createSystem` functionality already attached to the combined `Structure`. We ask for a reasonable cutoff, constrained hydrogen bonds (note that **this keyword argument overrides the fact that we use the `unconstrained` force field above**; the ligand (and all other molecules in the system) **will** have covalent bonds to hydrogen constrainted), PME, and rigid water:

In [ ]:
from simtk.openmm import app, unit, LangevinIntegrator
import numpy as np
from parmed.openmm import NetCDFReporter


system = complex_structure.createSystem(nonbondedMethod=app.PME,
                                         nonbondedCutoff=9*unit.angstrom,
                                         constraints=app.HBonds,
                                         rigidWater=True)


Next we'll set up the integrator, a reporter to write the trajectory, pick the timestep, and then go on to minimize the energy and run a very short amount of dynamics after setting the temperature to 300K:

In [ ]:
integrator = LangevinIntegrator(300*unit.kelvin, 
                                1/unit.picosecond, 
                                0.001*unit.picoseconds)
simulation = app.Simulation(complex_structure.topology, system, integrator)

# Depending on where your system came from, you may want to 
# add something like (30, 30, 30)*Angstrom to center the protein 
# (no functional effect, just visualizes better)
#simulation.context.setPositions(complex_structure.positions + np.array([30, 30, 30])*unit.angstrom)

simulation.context.setPositions(complex_structure.positions)

nc_reporter = NetCDFReporter('trajectory.nc', 10)
simulation.reporters.append(nc_reporter)

In [ ]:
simulation.minimizeEnergy()
minimized_coords = simulation.context.getState(getPositions=True).getPositions()

In [ ]:
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.step(1000)